##### PyTorch  : NN Module and Optim Module

In [78]:
# Create model class
import torch 
import torch.nn as nn

class Model(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.linear = nn.Linear(num_features, 1)
        self.sigmoid = nn.Sigmoid()


    def forward(self, features):
        out = self.linear(features)
        out = self.sigmoid(out)
        return out


In [79]:
# Create Dataset
features = torch.rand(10, 5)

#create a model
model = Model(features.shape[1])

# Call model forwardpass
# model.forward(features) this will work but the below if more good
model(features)

tensor([[0.5176],
        [0.4398],
        [0.4639],
        [0.4404],
        [0.4518],
        [0.4699],
        [0.5059],
        [0.4838],
        [0.4664],
        [0.4468]], grad_fn=<SigmoidBackward0>)

In [80]:
#Show model weights 
model.linear.weight

Parameter containing:
tensor([[ 0.1473, -0.2013, -0.1553, -0.3815,  0.1399]], requires_grad=True)

In [81]:
model.linear.bias

Parameter containing:
tensor([0.1170], requires_grad=True)

In [82]:
!pip install torchinfo

In [83]:
from torchinfo import summary

summary(model, input_size = (10, 5))

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [10, 1]                   --
├─Linear: 1-1                            [10, 1]                   6
├─Sigmoid: 1-2                           [10, 1]                   --
Total params: 6
Trainable params: 6
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

In [84]:
# Now let's make 2 Linear layers and train the model 

import torch 
import torch.nn as nn


class Model(nn.Module):

    def __init__(self, num_features):
        super().__init__()
        self.linear1 = nn.Linear(num_features, 3)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(3, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, features):
        out = self.linear1(features)
        out = self.relu(out)
        out = self.linear2(out)
        out = self.sigmoid(out)
        return out

In [85]:
# Create dataset 
features = torch.rand(10, 5)

# Create Model 
model = Model(features.shape[1])

#call model for forward pass
model(features)

tensor([[0.5282],
        [0.5274],
        [0.5250],
        [0.5056],
        [0.5215],
        [0.5299],
        [0.5180],
        [0.5049],
        [0.5087],
        [0.5327]], grad_fn=<SigmoidBackward0>)

In [86]:
model.linear1.weight

Parameter containing:
tensor([[-0.2382, -0.2098, -0.3177, -0.0774,  0.4198],
        [ 0.3682,  0.2665, -0.1063, -0.0383,  0.1771],
        [-0.4048, -0.3409, -0.0534,  0.0975, -0.1078]], requires_grad=True)

In [87]:
model.linear2.weight

Parameter containing:
tensor([[-0.4179,  0.1576, -0.5140]], requires_grad=True)

In [88]:
from torchinfo import summary

summary(model, input_size=(10, 5))

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [10, 1]                   --
├─Linear: 1-1                            [10, 3]                   18
├─ReLU: 1-2                              [10, 3]                   --
├─Linear: 1-3                            [10, 1]                   4
├─Sigmoid: 1-4                           [10, 1]                   --
Total params: 22
Trainable params: 22
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

In [89]:
# Now we can make sequential container to make the code easy

import torch 
import torch.nn as nn

class Model(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(num_features, 3),
            nn.ReLU(),
            nn.Linear(3, 1),
            nn.Sigmoid()
        )

    def forward(self, features):
        out = self.network(features)
        return out

In [90]:
# Create dataset
features = torch.rand(10, 5)

# Create model
model = Model(features.shape[1])

# call model for for forward pass
# model.forward(features)
model(features)

tensor([[0.5937],
        [0.5673],
        [0.5734],
        [0.5710],
        [0.5797],
        [0.5847],
        [0.5931],
        [0.5915],
        [0.5670],
        [0.5739]], grad_fn=<SigmoidBackward0>)

##### Now train with built in loss function

In [91]:
import numpy as np
import pandas as pd
import torch 
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


In [92]:
# Reading the dataset
df = pd.read_csv("https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv")
df

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,...,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,...,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,...,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,...,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,...,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,926424,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,NaN
565,926682,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,NaN
566,926954,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,NaN
567,927241,M,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,NaN


In [93]:
df.shape

(569, 33)

In [94]:
df.drop(columns=['id', 'Unnamed: 32'], inplace = True)

In [95]:
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [96]:
#Train Test Split

X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2)

In [97]:
# Scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [98]:
X_train

array([[ 0.73315014, -0.20103514,  0.66472042, ..., -0.33256647,
        -0.79997312, -1.02874549],
       [-0.49885332, -0.61540772, -0.50963028, ..., -0.40445779,
        -0.43533747, -0.41921376],
       [-0.1577736 , -1.19131536, -0.16130956, ...,  0.32377181,
         0.5560157 ,  0.49882657],
       ...,
       [ 0.93603376,  0.68389612,  0.87893981, ...,  1.33149253,
         0.53810948, -0.44006616],
       [-0.53413743, -1.37157914, -0.51434311, ..., -0.47790184,
         0.94995243,  0.45337903],
       [-0.11366847,  0.95546233, -0.16687926, ..., -0.63519323,
        -0.39626937, -0.76996184]])

In [99]:
y_train

363    B
74     B
482    B
174    B
448    B
      ..
421    B
376    B
264    M
130    B
545    B
Name: diagnosis, Length: 455, dtype: object

In [100]:
# Label Encoding
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [101]:
y_train

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0,
       0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0,

In [102]:
# Numpy Arrays to PyTorch Tensors

X_train_tensors = torch.from_numpy(X_train)
X_test_tensors = torch.from_numpy(X_test)
y_train_tensors = torch.from_numpy(y_train)
y_test_tensors = torch.from_numpy(y_test)

In [103]:
X_train_tensors.shape

torch.Size([455, 30])

In [104]:
X_train_tensors.shape[1]

30

In [105]:
y_train_tensors.shape

torch.Size([455])

In [106]:
class MySimpleNN(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.linear = nn.Linear(num_features, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, features):
        out = self.linear(features)
        out = self.sigmoid(out)

        return out
    
    # Usually we will define loss within class but now we will use the built in one directly outside the class

In [107]:
learning_rate = 0.1
epochs = 30
loss_function = nn.BCELoss()

##### Training Pipeline

In [111]:
# create model
model = MySimpleNN(X_train_tensors.shape[1])

# define optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# define loop
for epoch in range(epochs):

  # forward pass
  y_pred = model(X_train_tensors)

  # loss calculate
  loss = loss_function(y_pred, y_train_tensor.view(-1,1))

  # clear gradients
  optimizer.zero_grad()

  # backward pass
  loss.backward()

  # parameters update
  optimizer.step()

  # print loss in each epoch
  print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')

RuntimeError: mat1 and mat2 must have the same dtype, but got Double and Float